<a href="https://colab.research.google.com/github/mohsensafa/TensorFlow/blob/master/Cat%26Dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Download the cat&dog dataset
#to see whats inside, download it in your own local machine and explore in it, it may help you undestad easier how iterate through the files and read them
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2019-01-01 22:45:46--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.219.32.233, 2600:1417:76:28f::e59, 2600:1417:76:291::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.219.32.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M  86.1MB/s    in 8.8s    

2019-01-01 22:46:00 (89.4 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [0]:
#import necessery libraries
#I also imported the google drive for convenience, so I could directly save my model in my google drive
#from zipfile import ZipFile
import cv2
#from os import walk
#import io
import numpy as np
import pickle
import tensorflow as tf
from tensorflow.keras.applications import vgg16
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, InputLayer
from tensorflow.keras.callbacks import TensorBoard
from google.colab import drive
#from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
import time

drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
#load the downloaded zip file and iterate thnrough all the files and folders in it, read them and lebel them respectively

image = []
label = []
def Readandlabel():
    with ZipFile('kagglecatsanddogs_3367a.zip') as myzip:
        for file in myzip.namelist():
            if (len(file.split('.')) > 1) and file.split('.')[1] == 'jpg':
                try:
                    zipp = myzip.read(file)
                    zip1 = io.BytesIO(zipp)
                    file1 = (np.asarray(bytearray(zip1.read()), dtype=np.uint8))
                    im = cv2.imdecode(file1, cv2.IMREAD_GRAYSCALE)
                    im_resize = cv2.resize(im, (100,100))
                    image.append(im_resize)
                    if file.split('/')[1] == 'Cat':
                        label.append(0)
                    elif file.split('/')[1] == 'Dog':
                        label.append(1)
                except:
                    print((file))
                    pass
Readandlabel()

X = np.array(image).reshape(-1, 100, 100 , 1)


#save them using the pickle for later use

pickle_out = open("/gdrive/My Drive/Python/X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("/gdrive/My Drive/Python/label.pickle","wb")
pickle.dump(label, pickle_out)
pickle_out.close()


In [0]:
pickle_in = open("/gdrive/My Drive/Python/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("/gdrive/My Drive/Python/label.pickle","rb")
label = pickle.load(pickle_in)



In [0]:
#create our own convolutional neural network
NAME = "Cats-vs-dogs-CNN-{}".format(int(time.time()))
X = X/255.0

model = Sequential()

model.add(Conv2D(256, (6, 6), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (4, 4)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(1))
model.add(Activation('sigmoid'))

tensorboard = TensorBoard(log_dir="/gdrive/My Drive/Python/logs/{}".format(NAME))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, label, batch_size=32, epochs=8, validation_split=0.3, callbacks=[tensorboard])

Train on 17462 samples, validate on 7484 samples
Epoch 1/8
17462/17462 [==============================] - 129s 7ms/step - loss: 0.6009 - acc: 0.7136 - val_loss: 1.1789 - val_acc: 6.6809e-04
Epoch 2/8
17462/17462 [==============================] - 125s 7ms/step - loss: 0.5575 - acc: 0.7223 - val_loss: 1.0718 - val_acc: 0.2619
Epoch 3/8
17462/17462 [==============================] - 125s 7ms/step - loss: 0.4983 - acc: 0.7642 - val_loss: 0.8479 - val_acc: 0.4405
Epoch 4/8
17462/17462 [==============================] - 125s 7ms/step - loss: 0.4529 - acc: 0.7907 - val_loss: 1.0267 - val_acc: 0.4152
Epoch 5/8
17462/17462 [==============================] - 125s 7ms/step - loss: 0.4026 - acc: 0.8176 - val_loss: 0.7794 - val_acc: 0.5730
Epoch 6/8
17462/17462 [==============================] - 125s 7ms/step - loss: 0.3666 - acc: 0.8367 - val_loss: 1.1872 - val_acc: 0.4205
Epoch 7/8
17462/17462 [==============================] - 125s 7ms/step - loss: 0.3249 - acc: 0.8603 - val_loss: 1.2314 - val_

In [0]:
#in here basically we created a for loop to find the best model amoung the models we have in mind

pickle_in = open("/gdrive/My Drive/Python/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("/gdrive/My Drive/Python/label.pickle","rb")
label = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0, 1, 2]
layer_sizes = [64, 128, 256]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (6, 6), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="/gdrive/My Drive/Python/logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

            model.fit(X, label, batch_size=32, epochs=10, validation_split=0.15, callbacks=[tensorboard])

1-conv-64-nodes-0-dense-1546085367
Train on 21204 samples, validate on 3742 samples
Epoch 1/10
21204/21204 [==============================] - 17s 788us/step - loss: 0.6384 - acc: 0.6390 - val_loss: 0.7230 - val_acc: 0.5526
Epoch 2/10
21204/21204 [==============================] - 14s 667us/step - loss: 0.5712 - acc: 0.7054 - val_loss: 0.8008 - val_acc: 0.5171
Epoch 3/10
21204/21204 [==============================] - 14s 668us/step - loss: 0.5273 - acc: 0.7382 - val_loss: 0.8506 - val_acc: 0.4947
Epoch 4/10
21204/21204 [==============================] - 14s 670us/step - loss: 0.4877 - acc: 0.7685 - val_loss: 0.7693 - val_acc: 0.5951
Epoch 5/10
21204/21204 [==============================] - 14s 669us/step - loss: 0.4538 - acc: 0.7878 - val_loss: 0.8692 - val_acc: 0.5334
Epoch 6/10
21204/21204 [==============================] - 14s 669us/step - loss: 0.4167 - acc: 0.8100 - val_loss: 0.6692 - val_acc: 0.6790
Epoch 7/10
21204/21204 [==============================] - 14s 668us/step - loss: 0

In [0]:
#to use tensorboard in colab, I used ngrok to create a public link

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

LOG_DIR = '/gdrive/My\ Drive/Python/logs/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://e8150a18.ngrok.io


In [0]:
model.save('/gdrive/My Drive/Python/6epochs.model')

In [0]:
import cv2
import tensorflow as tf

CATEGORIES = ["Cat", "Dog"]


def prepare(filepath):
    IMG_SIZE = 100  
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE) 
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) 
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [0]:
start_time = time.time()
prediction = model.predict([prepare('/gdrive/My Drive/Python/cat.jpg')])  
print(CATEGORIES[int(prediction[0][0])])
print(time.time() - start_time)

Cat
0.019332408905029297
